In [0]:
DATA_FILES = [
    # '/datascope/subaru/data/targeting/MW/S25A_May-June/ga_targets_outerdisk_l90_b28_faint.ecsv',
    # '/datascope/subaru/data/targeting/MW/S25A_May-June/ga_targets_outerdisk_l90_b29_faint.ecsv',

    # '/datascope/subaru/data/targeting/MW/outerdisk_l90_bm28_SSP/ga_targets_outerdisk_l90_bm28_faint.ecsv',
    # '/datascope/subaru/data/targeting/MW/outerdisk_l90_bm29_SSP/ga_targets_outerdisk_l90_bm29_faint.ecsv',

    '/datascope/subaru/data/targeting/MW/outerdisk_l90_b25_SSP/ga_targets_outerdisk_l90_b25_faint.ecsv',
    '/datascope/subaru/data/targeting/MW/outerdisk_l90_b27_SSP/ga_targets_outerdisk_l90_b27_faint.ecsv',
]

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
if 'debug' not in globals():
    import debugpy
    debugpy.listen(('localhost', 5698))
    debug = True

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from pfs.ga.targeting.data import Observation
from pfs.ga.targeting.io import ObservationSerializer

from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI
from pfs.ga.targeting.diagram import FOV, FP

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord

In [0]:
# Load the data files

obs = []
for i, f in enumerate(DATA_FILES):
    s = ObservationSerializer(catalog_name='cat_{i + 1}')
    s.column_map = {
        'objid': 'targetid',
        'ra': 'RA',
        'dec': 'Dec'
    }
    obs.append(s.read(f))

In [0]:
obs[0].data.columns

In [0]:
obs[1].data.columns

In [0]:
obs[0].data[['psf_flux_r', 'r_ps1']]

In [0]:
obs[1].data.input_catalogs.unique()

In [0]:
obs[0].data['priority'].unique()

In [0]:
for i in range(len(obs)):
    plt.plot(*obs[i].get_coords(), '.')

### Plot the pointings

In [0]:
# l, b = 90.0, 28.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

# l, b = 90.0, 29.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

# l, b = 90.0, -28.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

# l, b = 90.0, -29.0
# gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
# print(gal.icrs)

l, b = 90.0, 25.0
gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
print(gal.icrs)

l, b = 90.0, 27.0
gal = SkyCoord(l=l * u.deg, b=b * u.deg, frame='galactic')
print(gal.icrs)

In [0]:
# pointing = Pointing(273.5, 60.9, posang=0.0)
# print(pointing.ra, pointing.dec)

# pointing = Pointing(340.5, 26.0, posang=0.0)
# print(pointing.ra, pointing.dec)

pointing = Pointing(279.5, 60.5, posang=0.0)
print(pointing.ra, pointing.dec)

wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(wcs.pointing)
fov = FOV(projection=wcs)

In [0]:
# fov.axes[0].limits = (190, 295)
# fov.axes[1].limits = (29, 25)

In [0]:
pointings = [
    # Pointing(273.525, 60.88, posang=0),
    # Pointing(270.725, 61.00, posang=0),

    # Pointing(340.4, 26.50, posang=0),
    # Pointing(341.215, 25.35, posang=0),

    Pointing(279.0, 60.45, posang=0),
    Pointing(276.3, 60.7, posang=0),
]

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240, subplot_kw=dict(projection=fov.projection.wcs))

fov.plot_catalog(ax, obs[0], c='red', size=0.5) #, scalex=False, scaley=False)
fov.plot_catalog(ax, obs[1], c='blue', size=0.5) #, scalex=False, scaley=False)

for p in pointings:
    pfi = SubaruPFI(SubaruWFC(p))
    pfi.plot_focal_plane(ax, fov, corners=True, color='k', lw=2, scalex=False, scaley=False)

ax.grid()